In [1]:
import enkie
import pta

/home/laurens/miniconda3/envs/thermo_pta/lib/python3.11/site-packages/enkie/compartment_parameters.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
import os
print(os.environ.get('GUROBI_HOME'))
print(os.environ.get('LD_LIBRARY_PATH'))


/home/laurens/gurobi/gurobi1300/linux64
/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server::/home/laurens/gurobi/gurobi1300/linux64/lib:/home/laurens/gurobi/gurobi1300/linux64/lib::/home/laurens/gurobi/gurobi1300/linux64/lib:/home/laurens/gurobi/gurobi1300/linux64/lib


In [2]:
model = pta.load_example_model("e_coli_core")
model.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = 0.5

Set parameter Username
Set parameter LicenseID to value 2744104
Academic license - for non-commercial use only - expires 2026-11-24


In [3]:
pta.prepare_for_pta(model)

Read LP format model from file /tmp/tmpvxxx309v.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


Error running the efmtool Java process:
Traceback (most recent call last):
  File "/home/laurens/miniconda3/envs/thermo_pta/lib/python3.11/site-packages/efmtool/efmtool.py", line 143, in _efmtool_process
    jpype.startJVM(*jvm_options, classpath=[libs_path])
  File "/home/laurens/miniconda3/envs/thermo_pta/lib/python3.11/site-packages/jpype/_core.py", line 267, in startJVM
    jvmpath = getDefaultJVMPath()
              ^^^^^^^^^^^^^^^^^^^
  File "/home/laurens/miniconda3/envs/thermo_pta/lib/python3.11/site-packages/jpype/_jvmfinder.py", line 70, in getDefaultJVMPath
    return finder.get_jvm_path()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/laurens/miniconda3/envs/thermo_pta/lib/python3.11/site-packages/jpype/_jvmfinder.py", line 204, in get_jvm_path
    raise JVMNotFoundException("No JVM shared library file ({0}) "
jpype._jvmfinder.JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.

EFM computation fail

RuntimeError: Error running the efmtool Java process

In [ ]:
parameters = enkie.CompartmentParameters.load('e_coli')
concentrations = pta.ConcentrationsPrior.load('M9_aerobic')
concentrations.add(pta.ConcentrationsPrior.load('ecoli_M9_glc'))

thermodynamic_space = pta.ThermodynamicSpace.from_cobrapy_model(
    model,
    metabolites_namespace = "bigg.metabolite",
    parameters=parameters,
    concentrations=concentrations
)

In [ ]:
problem = pta.PmoProblem(model, thermodynamic_space, solver='GUROBI')
problem.solve()

In [ ]:
assessment = pta.QuantitativeAssessment(problem)
assessment.summary()